In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 0c995391-9e19-4475-84e7-f4cdd45b3244, 3, Finished, Available)

In [2]:
from pyspark.sql import SparkSession

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 4, Finished, Available)

In [3]:
spark = SparkSession.builder \
    .appName("Data Merge") \
    .getOrCreate()

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 5, Finished, Available)

In [4]:
user_activity_df = spark.sql("SELECT *, timestamp AS user_activity_timestamp FROM datasets.user_activity_data")
display(user_activity_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, a7a08284-d8e1-4d0f-9693-da100b57412a)

In [5]:
recommendation_df = spark.sql("SELECT * FROM datasets.recommendation_data LIMIT 1000")
display(recommendation_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 32cfa39f-d62e-43fe-bff6-4083113399d6)

In [6]:
engagement_type_df = spark.sql("SELECT *, timestamp AS engagement_timestamp FROM datasets.engagementtype_data ")
display(engagement_type_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 11de0f3c-65a0-4db6-80fb-baa36943f2f4)

In [7]:
nearby_place_df = spark.sql("SELECT * FROM datasets.nearby_place_data LIMIT 1000")
display(nearby_place_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, f0b96586-7779-4e6f-b95a-561a445b8637)

In [8]:
merged_df1 = user_activity_df.join(recommendation_df, on='userID', how='left')

merged_df2 = merged_df1.join(nearby_place_df, on='postID', how='left')

final_df = merged_df2.join(engagement_type_df, 
                           (merged_df2.userID == engagement_type_df.userID) & 
                           (merged_df2.postID == engagement_type_df.postID), 
                           how='left')

final_df = final_df.drop(engagement_type_df.userID).drop(engagement_type_df.postID)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 10, Finished, Available)

In [9]:
display(final_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5f9a8d24-9ed0-4c75-8e02-0e2982e635b5)

In [10]:
from pyspark.sql import SparkSession
final_df.createOrReplaceTempView("final_data")


StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 12, Finished, Available)

In [11]:
unique_activity_ids_df = spark.sql("""
SELECT DISTINCT activityID FROM final_data
""")

unique_activity_ids_df.show()


StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 13, Finished, Available)

+--------------------+
|          activityID|
+--------------------+
|      Profile Update|
|         Access Page|
|              Logout|
|               Share|
|             Comment|
|               Login|
|       Like/Reaction|
|Notification Inte...|
|              Search|
|       Post Creation|
|     Follow/Unfollow|
+--------------------+



In [12]:

# 1.（DAU）
dau_df = spark.sql("""
    SELECT 
        DATE(user_activity_timestamp) AS event_date,
        COUNT(DISTINCT userID) AS DAU
    FROM final_data
    WHERE activityID = 'Login'
    GROUP BY event_date
    ORDER BY event_date
""")

display(dau_df)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 88ae14a6-d3ea-4657-8af8-480cb5d60ae1)

In [13]:
# 2.（MAU）

mau_df = spark.sql("""
    SELECT 
        DATE_FORMAT(user_activity_timestamp, 'yyyy-MM') AS month,
        COUNT(DISTINCT userID) AS MAU
    FROM final_data
    GROUP BY month
    ORDER BY month
""")

display(mau_df)


StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, fa5f7d85-a093-41e7-aef8-c8c0ccc350ba)

In [14]:
# 3. day_1_retention
dau_df = spark.sql("""
SELECT 
    signup_date, 
    COUNT(DISTINCT r1.user_id) / COUNT(DISTINCT u.user_id) AS day_1_retention
FROM (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS signup_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) u
LEFT JOIN (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS return_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) r1 ON u.user_id = r1.user_id AND r1.return_date = DATE_ADD(u.signup_date, 1)
GROUP BY signup_date
ORDER BY signup_date;
""")


display(dau_df)



StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 85d18772-f01d-45d9-b62c-a18c32a3475d)

In [15]:
# 4.day_7_retention
dau_df = spark.sql("""
SELECT 
    signup_date, 
    COUNT(DISTINCT r7.user_id) / COUNT(DISTINCT u.user_id) AS day_7_retention
FROM (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS signup_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) u
LEFT JOIN (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS return_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) r7 ON u.user_id = r7.user_id AND r7.return_date = DATE_ADD(u.signup_date, 7)
GROUP BY signup_date
ORDER BY signup_date;
""")
display(dau_df)



StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2d014b65-d531-4192-8df1-4c0edce51e47)

StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 54ae2fcc-85e5-4c17-8d87-019927d35355)

In [17]:
# 5.day_30_retention

dau_df = spark.sql("""
SELECT 
    signup_date, 
    COUNT(DISTINCT r30.user_id) / COUNT(DISTINCT u.user_id) AS day_30_retention
FROM (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS signup_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) u
LEFT JOIN (
    SELECT 
        userID AS user_id, 
        DATE(user_activity_timestamp) AS return_date
    FROM final_data
    WHERE activityID IN ('Login', 'Access Page', 'Logout', 'Share', 'Comment', 'Like/Reaction', 'Notification Interaction', 'Post Creation', 'Search', 'Follow/Unfollow')
    GROUP BY userID, DATE(user_activity_timestamp)
) r30 ON u.user_id = r30.user_id AND r30.return_date = DATE_ADD(u.signup_date, 30)
GROUP BY signup_date
ORDER BY signup_date;
""")

display(dau_df)


StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7ea0b735-e664-435b-ad0e-4ef9180a8448)

In [18]:
#6. Event Interaction KPIsEvent Participation Rate: Percentage of users who participate in events out of the total number of users.

dau_df = spark.sql("""
SELECT 
    eventID,
    COUNT(DISTINCT userID) AS participating_users,
    COUNT(DISTINCT userID) / (SELECT COUNT(DISTINCT userID) FROM final_data) AS event_participation_rate
FROM final_data
GROUP BY eventID
ORDER BY eventID;
""")

display(dau_df)



StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, ca1336c9-1da4-446e-b9f7-86b49dfc64f5)

In [19]:
#7. Event Engagement:Likes, Comments, Shares: Number of likes, comments, and shares per event.

dau_df = spark.sql("""
SELECT 
    eventID,
    SUM(CASE WHEN engagementType = 'Like' THEN 1 ELSE 0 END) AS like_count,
    SUM(CASE WHEN engagementType = 'Save' THEN 1 ELSE 0 END) AS comment_count,
    SUM(CASE WHEN engagementType = 'Share' THEN 1 ELSE 0 END) AS share_count
FROM final_data
WHERE engagementType IN ('Like', 'Save', 'Share')
GROUP BY eventID;
""")

display(dau_df)



StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, a55b894c-3c00-4858-979f-2977f319dd92)

In [22]:
#8. Daily postings

dau_df = spark.sql("""
SELECT 
    'daily' AS period,
    DATE(user_activity_timestamp) AS date,
    COUNT(*) AS post_count
FROM final_data
WHERE activityID = 'Post Creation'
GROUP BY DATE(user_activity_timestamp)

UNION ALL

SELECT 
    'weekly' AS period,
    DATE_TRUNC('week', user_activity_timestamp) AS date,
    COUNT(*) AS post_count
FROM final_data
WHERE activityID = 'Post Creation'
GROUP BY DATE_TRUNC('week', user_activity_timestamp)

UNION ALL

SELECT 
    'monthly' AS period,
    DATE_TRUNC('month', user_activity_timestamp) AS date,
    COUNT(*) AS post_count
FROM final_data
WHERE activityID = 'Post Creation'
GROUP BY DATE_TRUNC('month', user_activity_timestamp)

ORDER BY period, date;
""")

display(dau_df)



StatementMeta(, 195e4b0f-83d6-4432-8447-247a352aa4e0, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, bb9bdf78-9dda-4bff-ab77-1e84a5a41043)